In [ ]:
# Starta med covering sets
# vi börjar enkelt med 5 matcher
# Eftersom vi tänker oss att vi delar in grupper av matcher
# Speciellt om vi är osäkra på utfallet för vissa, då kanske vi vill garantera

In [1]:
import itertools



In [62]:
#----------------------------------------------------
# Utvärdera hur stor chans vi har att vinna!
#----------------------------------------------------
# Som om varje enskilt utfall vore lika sannolikt...
# 
# Räkna hur många i all_combinations som får korrekt mha covering set...
# 

import itertools
from collections import Counter

def analyze_covering_set(covering_set, all_combinations):
    correct_counts = Counter()

    for combo in all_combinations:
        max_correct = 0
        for ticket in covering_set:
            correct = sum(a == b for a, b in zip(ticket, combo))
            max_correct = max(max_correct, correct)
        correct_counts[max_correct] += 1

    result = correct_counts
    for correct, count in sorted(result.items()):
        print(f"Combinations with {correct} correct: {count} {round(100*count/len(all_combinations), 2)}% ")

    return "Finished."

# Example usage
# covering_set = [
#     ('1', '1', '1', '1', '1'),
#     ('X', 'X', 'X', 'X', 'X'),
#     ('2', '2', '2', '2', '2'),
#     ('1', '1', '1', 'X', '2'),
#     ('X', 'X', '1', '2', '2')
# ]
# Vi använder covering_set från ovan istället!

# Generera alla kombinationer möjliga på 5 matcher
## all_combinations = list(itertools.product('1X2', repeat=5))
# Använd all combinations från ovan!

#result = analyze_covering_set(covering_set, all_combinations)

# Display the results
#for correct, count in sorted(result.items()):
#    print(f"Combinations with {correct} correct: {count} {round(100*count/len(all_combinations), 2)}% ")


In [60]:
# Fall nummer 1 : Det helt naiva sättet - 
# Vi räknar ut hur många av M matcher vi får 4 rätt på.

covering_set = []
all_combinations = list(itertools.product('1X2', repeat=5))

covered_combinations = set()

while len(covered_combinations) < len(all_combinations):
    best_ticket = None
    max_new_covered = 0
    
    # Generate a candidate ticket
    for candidate in all_combinations:
        new_covered = sum(1 for combo in all_combinations 
                          if combo not in covered_combinations 
                          and sum(a == b for a, b in zip(candidate, combo)) >= 4)
        
        if new_covered > max_new_covered:
            max_new_covered = new_covered
            best_ticket = candidate
    
    if best_ticket:
        covering_set.append(best_ticket)
        for combo in all_combinations:
            if sum(a == b for a, b in zip(best_ticket, combo)) >= 4:
                covered_combinations.add(combo)


In [61]:
# Utvärdera

In [63]:
analyze_covering_set(covering_set, all_combinations)

Combinations with 4 correct: 212 87.24% 
Combinations with 5 correct: 31 12.76% 


'Finished.'

In [ ]:
# Om vi har vissa utfall omöjliga så måste vi berätta det

In [64]:
# Fall nummer 2 : ta bort vissa utfall från några av matcherna.
# Generera rader som täcker in n av M möjliga.
# constrained = färre möjliga rader tas in från början.
# 

import itertools

def generate_constrained_combinations(game_constraints):
    return list(itertools.product(*[constraints for constraints in game_constraints]))

def generate_covering_set(game_constraints, guarantee):
    all_combinations = generate_constrained_combinations(game_constraints)
    covering_set = []
    covered_combinations = set()

    while len(covered_combinations) < len(all_combinations):
        best_ticket = None
        max_new_covered = 0
        # new_covered --> antal träffar som en ticket har som bäst mot alla kombinationer... men minus redan täckta kombinationer.
        for candidate in all_combinations:
            new_covered = sum(1 for combo in all_combinations 
                              if combo not in covered_combinations 
                              and sum(a == b for a, b in zip(candidate, combo)) >= guarantee)
            
            if new_covered > max_new_covered:
                max_new_covered = new_covered
                best_ticket = candidate

        # Behåll alla tickets som täcker tillräckligt mycket - dessa behöver vi inte jämföra mot igen...
        if best_ticket:
            covering_set.append(best_ticket)
            for combo in all_combinations:
                if sum(a == b for a, b in zip(best_ticket, combo)) >= guarantee:
                    covered_combinations.add(combo)
        else:
            break  # No more tickets can improve coverage

    return covering_set

# Example usage
game_constraints = [
    ('1', '2'),  # Game 1: Only 1 and 2 are possible
    ('1', 'X', '2'),  # Game 2: All outcomes possible
    ('1', 'X', '2'),  # Game 3: All outcomes possible
    ('1', 'X', '2'),  # Game 4: All outcomes possible
    ('1', 'X', '2'),  # Game 4: All outcomes possible
    ('1', 'X', '2'),  # Game 4: All outcomes possible
    ('1', 'X')   #
]
all_combinations =  generate_constrained_combinations(game_constraints)



covering_set = generate_covering_set(game_constraints, 6)
print(f"Number of tickets in covering set: {len(covering_set)}")
for ticket in covering_set[:5]:
    print(ticket)

analyze_covering_set(covering_set, all_combinations)


Number of tickets in covering set: 118
('1', '1', '1', '1', '1', '1', '1')
('1', '1', '1', '1', 'X', 'X', 'X')
('1', '1', '1', 'X', '1', '2', 'X')
('1', '1', '1', 'X', '2', 'X', '1')
('1', '1', '1', '2', 'X', '2', '1')
Combinations with 6 correct: 854 87.86% 
Combinations with 7 correct: 118 12.14% 


'Finished.'

In [56]:
all_combinations =  generate_constrained_combinations(game_constraints)


In [58]:
len(all_combinations)

972

In [2]:
print(123+15+87+18)

243


In [39]:
import random
from collections import Counter

# Vi matar in odds, men behöver konvertera till sannolikheter.
def convert_odds_to_probabilities(odds):
    probabilities = []
    for match in odds:
        total = sum(1/odd for odd in match)
        probabilities.append(tuple((1/odd)/total for odd in match))
    return probabilities

# Function to generate outcome combinations based on individual match probabilities.
def generate_probable_outcomes(match_probabilities, num_samples=10000):
    # Each element in match_probabilities is a tuple specifying weights for ['1', 'X', '2']
    outcomes = []
    num_matches = len(match_probabilities)
    for _ in range(num_samples):
        outcome = tuple(random.choices(['1', 'X', '2'], weights=match_probabilities[i], k=1)[0]
                        for i in range(num_matches))
        outcomes.append(outcome)
    # Optionally deduplicate if many repeated outcomes are expected
    return list(set(outcomes))

# Greedy algorithm for building the covering set.
def generate_covering_set(probable_combinations, guarantee):
    covering_set = []
    covered_combinations = set()
    
    # Continue until every outcome in the probable sample is covered.
    while len(covered_combinations) < len(probable_combinations):
        best_ticket = None
        max_new_covered = 0
        
        # Evaluate each candidate ticket from the probable outcomes
        for candidate in probable_combinations:
            new_covered = sum(
                1 for outcome in probable_combinations
                if outcome not in covered_combinations and sum(a == b for a, b in zip(candidate, outcome)) >= guarantee
            )
            
            if new_covered > max_new_covered:
                max_new_covered = new_covered
                best_ticket = candidate
        
        if best_ticket:
            covering_set.append(best_ticket)
            for outcome in probable_combinations:
                if sum(a == b for a, b in zip(best_ticket, outcome)) >= guarantee:
                    covered_combinations.add(outcome)
        else:
            break  # No candidate can add new coverage; exit loop.
    return covering_set


# Utvärdera om dom genererade outcomes, faktiskt innehåller sant utfall
def count_true_outcome_matches(true_tuple, generated_list):
    return sum(1 for generated_tuple in generated_list if generated_tuple == true_tuple)

# Example usage:
# Assume each match has probabilities. For example, for 15 matches, you may have:
# Each tuple corresponds to the weights for outcomes '1', 'X', '2' respectively.

# Vi tar lördag 15/2, 2025 , vecka 7
match_odds = [
    ( 1.96, 3.14, 4.46  ), # Crystal Palace - Everton
    (2.04, 3.94, 3.25  ),  # Man City  - Newcastle
    (1.41, 4.75, 7.09  ), # Aston Villa - Ipswich
    (2.14, 3.36, 3.46   ),  # Fulham - Nottingham
    (2.35, 3.59, 2.88   ), # West Ham - Brentford
    (5.86 , 4.51 , 1.50 ), # Southampton - Bournemouth
    (1.83 , 3.56 , 4.33 ), # Blackburn - Plymouth
    ( 2.42, 3.34 , 2.87 ), # Hull - Norwich
    (2.80 , 3.21 , 2.55 ), # Luton - Sheffield U https://www.oddsportal.com/football/england/premier-league/west-ham-brentford-CMnhLYoL/
    ( 1.59, 4.04 , 5.29 ), # Middlesbrough - Watford
    ( 2.78, 2.96 , 2.78 ), # Millwall - W.B.A.
    ( 2.32, 3.34 , 3.04 ), # Sheffield Wednesday - Coventry
    ( 2.39, 3.28 , 2.98 ), # Stoke - Swansea    
]


match_probabilities = convert_odds_to_probabilities(match_odds)
## print(match_probabilities[0])

# Generate the 10,000 most probable outcomes.
probable_combinations = generate_probable_outcomes(match_probabilities, num_samples=10)
print(probable_combinations[0])

('X', 'X', '1', '1', '1', '2', '1', '2', 'X', '1', 'X', 'X', '1')


In [40]:
def print_tuple_list(tuple_list, items_per_line=1):
    for i, t in enumerate(tuple_list):
        print(f"{i+1}. {' '.join(t)}")
        if (i + 1) % items_per_line == 0:
            print()

print_tuple_list(probable_combinations)

1. X X 1 1 1 2 1 2 X 1 X X 1

2. X X 1 X X 2 1 1 2 1 1 2 2

3. 1 1 1 X X 1 X X 2 1 1 1 1

4. 2 1 1 1 1 2 1 1 2 2 2 X 2

5. X 2 1 2 2 2 1 X X 1 X 1 X

6. 2 1 X 1 X 2 X 2 X 2 1 1 1

7. 2 2 1 2 2 2 X 1 X 1 X X X

8. 1 2 X X X 2 1 1 2 2 2 1 2

9. X 2 X 1 1 X X X 2 X 1 X X

10. 1 2 1 2 2 2 1 X 2 X 2 X X



In [35]:
true_outcome = ('2', '1', 'X', '1', '2', '2', '1', 'X', '2', '2', 'X', '2', '1')
# true_outcome = probable_combinations[0] ### fuska och sätt en befintlig genererad rad till den sanna raden

true_generated_combos = count_true_outcome_matches(true_outcome, probable_combinations)
print(true_generated_combos)
break
# Build the covering set with a guarantee of, say, 13 correct predictions.
covering_set = generate_covering_set(probable_combinations, guarantee=13)

print("Number of tickets in covering set:", len(covering_set))
# Optionally, analyze how well the covering set works against the generated probable outcomes.

0


SyntaxError: 'break' outside loop (320387668.py, line 6)

In [24]:
print(probable_combinations[0] == true_outcome)

False


In [30]:
print(probable_combinations[4])

('1', '1', 'X', '1', 'X', '2', '1', '1', 'X', 'X', '1', '2', '2')


In [26]:
print(len(probable_combinations))

10


In [27]:
probable_combinations == true_outcome

False